In [14]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
import kagglehub
from torch.utils.data import Dataset
import pandas as pd

In [21]:
path = kagglehub.dataset_download("atulyakumar98/pothole-detection-dataset")

100%|██████████| 194M/194M [00:09<00:00, 21.9MB/s] 

Extracting files...


In [22]:
print("Dataset downloaded to:", path)

Dataset downloaded to: /Users/sai/.cache/kagglehub/datasets/atulyakumar98/pothole-detection-dataset/versions/4


In [24]:
class PotholeDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.image_names = os.listdir(image_dir)
        self.transform = transform

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_dir, self.image_names[idx])
        image = Image.open(image_path).convert("RGB")
        
        label = 0 if 'normal' in self.image_names[idx] else 1
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor(),
])